In [1]:
%load_ext cuml.accel
%load_ext cudf.pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

cuML: Accelerator installed.


In [2]:
# #-----------------------------------------------------
# ## convert to parquet
# # مسیر فایل CSV اولیه
# csv_path = "AnnonymData.csv"
# # مسیر خروجی Parquet
# parquet_path = "AnnonymData.parquet"
#
# # یک‌بار: تبدیل CSV به Parquet
# gdf_initial = pd.read_csv(csv_path)
# gdf_initial.to_parquet(parquet_path, compression="snappy")
# #-----------------------------------------------------


In [3]:
data = pd.read_parquet('AnnonymData.parquet')
df =pd.DataFrame(data)
df = df.drop(columns=['OrderId' , 'TransactionId','BookingNr'] , axis=1)
print( 'Data frame dimensions: ',df.shape)
df.head(3)

Data frame dimensions:  (6538739, 11)


,DateOfService,DateOfOrder,OrderQty,MenuName,MenuPrice,MenuSubsidy,GroupName,CanceledQty,DateOfCancel,Site,SchoolID
0,2020-01-02,2020-02-05 11:54:08,1,Mittagessen (Gs),"3,10","0,00","xxx3,45€ normal 5T (68€)",0,<NA>,LP,SCH001
1,2020-01-02,2019-12-16 10:30:51,1,Smart Eating Buffet (WGrus),"0,00","3,50",Steinfurt Abo ermäßigt,0,<NA>,BK,SCH002
2,2020-01-02,2019-12-16 10:31:33,1,Smart Eating Buffet (WGrus),"2,90","0,60",Westerkappeln Grundschüler Abo,0,<NA>,BK,SCH002


In [4]:
df.columns

Index(['DateOfService', 'DateOfOrder', 'OrderQty', 'MenuName', 'MenuPrice',
       'MenuSubsidy', 'GroupName', 'CanceledQty', 'DateOfCancel', 'Site',
       'SchoolID'],
      dtype='object')

In [5]:
missing_data = df.isna().sum()
missing_data = missing_data[missing_data > 0]
print(missing_data)

DateOfCancel    5604260
Site             846012
dtype: int64


If the value of the DateOfCancel column in a row is null, is the value of the CanceledQty column also zero in the same row? If not, return the rows that do not meet this condition.

In [6]:
# violations = df[df['DateOfCancel'].isnull() & (df['CanceledQty'] != 0)]
# print(violations)

In [7]:

df['DateOfService']  = pd.to_datetime(df['DateOfService'] ,errors='coerce')
df['DateOfOrder']    = pd.to_datetime(df['DateOfOrder'], errors='coerce')
df['DateOfCancel']   = pd.to_datetime(df['DateOfCancel'], errors='coerce')

df['MenuPrice'] = pd.to_numeric(df['MenuPrice'].str.replace(',', '.'), errors='coerce').astype(float)
df['MenuSubsidy'] = pd.to_numeric(df['MenuSubsidy'].str.replace(',', '.'), errors='coerce').astype(float)
df['OrderQty'] = df['OrderQty'].astype(int)
df['CanceledQty'] = df['CanceledQty'].astype(int)

# df['IsCanceled'] = df['DateOfCancel'].notna().astype(int)  # 1 اگر کنسلی شده (تاریخ داره)، 0 اگر نه (NaN)

# فقط ردیف‌هایی را نگه می‌داریم که تعداد سفارش بزرگتر یا مساوی تعداد لغو شده باشد
df = df[df['OrderQty'] >= df['CanceledQty']]

# ردیف‌ها را در دو حالت نگه می‌داریم:
# 1. سفارش اصلاً لغو نشده باشد (DateOfCancel خالی است)
# 2. یا اگر لغو شده، تاریخ لغو آن قبل یا مساوی تاریخ سرویس باشد
df = df[(df['DateOfCancel'].isna()) | (df['DateOfCancel'] <= df['DateOfService'])]

# تاریخ سرویس باید از تاریخ سفارش بزرگتر باشد
df = df[df['DateOfService'] >= df['DateOfOrder']]


df = df.dropna(subset=['Site'])



# df = df.drop(columns=['DateOfCancel'])
df = df.sort_values(by='DateOfOrder', ascending=True)

df = df.reset_index(drop=True)
print( 'Data frame dimensions: ',df.shape)
df.head(5)


Data frame dimensions:  (3888273, 11)


,DateOfService,DateOfOrder,OrderQty,MenuName,MenuPrice,MenuSubsidy,GroupName,CanceledQty,DateOfCancel,Site,SchoolID
0,2020-01-02,2019-12-02 20:39:46,1,Mittagessen (BS),3.05,0.0,Bad Sassendorf,0,NaT,LP,SCH003
1,2020-01-03,2019-12-02 20:39:46,1,Mittagessen (BS),3.05,0.0,Bad Sassendorf,0,NaT,LP,SCH003
2,2020-01-02,2019-12-03 22:29:24,1,Mittagessen (BS),3.05,0.0,Bad Sassendorf,0,NaT,LP,SCH003
3,2020-01-02,2019-12-04 13:27:22,1,Mittagessen (BS),3.05,0.0,Bad Sassendorf,0,NaT,LP,SCH003
4,2020-01-03,2019-12-04 13:27:22,1,Mittagessen (BS),3.05,0.0,Bad Sassendorf,0,NaT,LP,SCH003


In [8]:
def show_unique_values(df, column_name, excel_output=None):
    if column_name in df.columns:
        unique_values = df[column_name].value_counts()
        print(f"Unique values in column '{column_name}':")
        print(unique_values)

        # Save to Excel if output path is provided
        if excel_output:
            # Convert to DataFrame for better Excel formatting
            unique_df = pd.DataFrame({
                'Value': unique_values.index,
                'Count': unique_values.values
            })
            unique_df.to_excel(excel_output, index=False)
            print(f"\nResults saved to Excel file: {excel_output}")
    else:
        print(f"There is no column with name '{column_name}' in the data frame.")

# استفاده از تابع برای ستون 'Name'
show_unique_values(df, "MenuName" , excel_output="menu_name.xlsx")

Unique values in column 'MenuName':
MenuName
Mittagessen Outlaw                226158
Smart Eating Buffet (HGS)         181046
Mittagessen (KL)                  171590
Smart Eating Buffet (LSS)         148734
Smart Eating Buffet (WGrus)       139884
                                   ...  
Menü A (BO-Mitte)                      2
Menü B1 (BO-Mitte)                     2
Menü A (FSC)                           1
Menü B2 Ohne Dessert (BOHilde)         1
Menü C (GSZ)                           1
Name: count, Length: 269, dtype: int64

Results saved to Excel file: menu_name.xlsx


In [9]:
import re


def clean_text_menu(s):
    s = s.strip()
    s = re.sub(r'\s+', ' ', s)             # حذف فاصله‌های اضافی
    s = re.sub(r'[-–—]', ' ', s)            # یکدست کردن خط تیره
    s = re.sub(r'\s*\([^)]*\)', '', s)         # حذف پرانتز انتهایی
    s = re.sub(r'[()]', '', s)                  # حذف پرانتز های مانده
    s = s.replace('.', '')                 # ← حذف تمام نقاط


    # اصلاح املا
    s = re.sub(r'^Menü A', 'Menu A', s, flags=re.I)
    s = re.sub(r'^Menue_A', 'Menu A', s, flags=re.I)
    s = re.sub(r'^Menü B', 'Menu B', s, flags=re.I)
    s = re.sub(r'^Menue_B1', 'Menu B1', s, flags=re.I)
    s = re.sub(r'^Menü B1', 'Menu B1', s, flags=re.I)
    s = re.sub(r'^Menü B1 veg', 'Menu B1 veg', s, flags=re.I)
    s = re.sub(r'^Menü veg B1', 'Menu B1 veg', s, flags=re.I)
    s = re.sub(r'^Menü B1 Ohne Dessert', 'Menu B1 Ohne Dessert', s, flags=re.I)
    s = re.sub(r'^Menü C', 'Menu C', s, flags=re.I)
    s = re.sub(r'^MENÜ C', 'Menu C', s, flags=re.I)
    s = re.sub(r'^Menü B2', 'Menu B2', s, flags=re.I)
    s = re.sub(r'^Menue_B2', 'Menu B2', s, flags=re.I)
    s = re.sub(r'^Menü B2 Ohne Dessert', 'Menu B2 Ohne Dessert', s, flags=re.I)
    s = re.sub(r'^Menu B2BRS', 'Menu B2', s, flags=re.I)

    s = re.sub(r'^Menü D', 'Menu D', s, flags=re.I)
    s = re.sub(r'^DGE MENÜ', 'Menu DGE', s, flags=re.I)
    s = re.sub(r'^DGE Menü', 'Menu DGE', s, flags=re.I)
    s = re.sub(r'^Musterlinie 1', 'Musterlinie', s, flags=re.I)
    s = re.sub(r'^veganes Menülinie', 'veganes Menu', s, flags=re.I)
    s = re.sub(r'^veganes Menü', 'veganes Menu', s, flags=re.I)


    return s.strip()

df['MenuName'] = df['MenuName'].astype(str).apply(clean_text_menu)

show_unique_values(df, "MenuName" , excel_output="menu_name_cleaned.xlsx")

Unique values in column 'MenuName':
MenuName
Smart Eating Buffet          2460699
Mittagessen                   401310
Menu A                        317169
Mittagessen Outlaw            226158
Menu DGE                      149262
Menu B1 veg                    78482
Menu B1                        67196
Menu C                         36974
Tagesmenü                      34351
Menu B2                        30633
Riesenbeck                     17541
Nudelbar                       16586
Kiga FZ Nicolai Lippstadt      13039
Salatbar                       10650
Smart Eating                    7837
Lunchpaket                      7250
Menu A Ohne Dessert             5103
Menu B1 Ohne Dessert            3741
Lunchpaket vegetarisch          1584
Menu D                          1380
Menu B                           889
Menu B2 Ohne Dessert             302
veganes Menu                     131
Musterlinie                        6
Name: count, dtype: int64

Results saved to Excel file: menu_name_c

In [10]:
def show_unique_values(df, column_name, excel_output=None):
    if column_name in df.columns:
        unique_values = df[column_name].value_counts()
        print(f"Unique values in column '{column_name}':")
        print(unique_values)

        # Save to Excel if output path is provided
        if excel_output:
            # Convert to DataFrame for better Excel formatting
            unique_df = pd.DataFrame({
                'Value': unique_values.index,
                'Count': unique_values.values
            })
            unique_df.to_excel(excel_output, index=False)
            print(f"\nResults saved to Excel file: {excel_output}")
    else:
        print(f"There is no column with name '{column_name}' in the data frame.")

# استفاده از تابع برای ستون 'Name'
show_unique_values(df, "GroupName" , excel_output="Group_name.xlsx")

Unique values in column 'GroupName':
GroupName
Steinfurt Abo ermäßigt         194774
Bochum Schüler                 170965
Emsdetten Abo Schüler          163331
Outlaw Kita                    162306
Greven Schüler Abo             146807
                                ...  
Marienschule                        2
Herten Lehrer                       1
Oelde Lehrer                        1
Westerkappeln Lehrer                1
xxx3,20€ normal 4T (42,40€)         1
Name: count, Length: 217, dtype: int64

Results saved to Excel file: Group_name.xlsx


In [11]:
import re


def clean_text_group(s: str) -> str:
    """
    این تابع یک نام گروه را به عنوان ورودی دریافت کرده و بر اساس قوانین از پیش تعریف شده،
    آن را پاک‌سازی و به یک دسته استاندارد تبدیل می‌کند. این تابع مکان‌ها، انواع گروه‌ها
    (دانش‌آموز، معلم)، برنامه‌های خاص (BuT) و سایر کلمات کلیدی را شناسایی می‌کند.
    """

    # 1. پیش‌پردازش و استانداردسازی اولیه
    text = s.strip()
    text = re.sub(r'[-–—]', ' ', text)      # یکدست کردن خط تیره
    text = re.sub(r'\s*\([^)]*\)', '', text)  # حذف محتوای داخل پرانتز
    text = re.sub(r'[()]', '', text)         # حذف خود پرانتز
    text = text.replace('.', '')             # حذف نقاط
    text = re.sub(r'\s+', ' ', text)         # حذف فاصله‌های اضافی
    text = text.lower()                      # تبدیل به حروف کوچک برای مقایسه بهتر

    # حذف پیشوندهای فنی مانند xx یا xxx
    text = re.sub(r'^x{2,3}', '', text).strip()

    # اصلاح املای کلمات رایج قبل از بررسی
    if ' erm' in text: # فاصله برای جلوگیری از جایگزینی در کلماتی مثل werne
        text = text.replace(' erm', ' ermäßigt')
    text = text.replace('grundsch', 'grundschule')
    text = text.replace('kiga', 'kita')

    # --- 2. دسته‌بندی بر اساس قوانین ---

    # قانون اولویت بالا: برنامه‌های خاص
    if 'but' in text:
        return "Programm: BuT"
    if 'corona' in text:
        return "Programm: Corona"
    if 'primus' in text:
        return "Programm: Primus"

    # قانون اصلی: دسته‌بندی بر اساس مکان (شهر)
    cities = [
        'arnsberg', 'bad sassendorf', 'beckum', 'neubeckum', 'bergkamen', 'bielefeld', 'bochum',
        'bönen', 'borghorst', 'borken', 'datteln', 'dortmund', 'dülmen', 'emsdetten', 'enger',
        'ennigerloh', 'essen', 'gelsenkirchen', 'gladbeck', 'greven', 'gütersloh', 'halle',
        'hamm', 'herford', 'herne', 'herten', 'hörstel', 'riesenbeck', 'horstmar', 'kreis soest', 'soest',
        'lippe', 'lippetal', 'lippstadt', 'lüdinghausen', 'märkischer kreis', 'münster',
        'nordkirchen', 'oelde', 'olfen', 'ostbevern', 'recklinghausen', 'senden', 'sprockhövel',
        'steinfurt', 'unna', 'waltrop', 'warendorf', 'werl', 'werne', 'wersen', 'wesel',
        'westerkappeln', 'wetter', 'wickede'
    ]

    found_city = None
    for city in cities:
        if city in text:
            found_city = city.title()
            break

    if found_city:
        # اگر شهری پیدا شد، بقیه دسته‌بندی را بر اساس آن می‌سازیم
        category_parts = []

        # شناسایی نوع اصلی گروه
        if 'schüler' in text or 'grundschule' in text:
            category_parts.append("Schüler")
        elif 'lehrer' in text:
            category_parts.append("Lehrer")
        elif 'kita' in text:
            category_parts.append("Kita")
        elif 'jobcenter' in text:
            category_parts.append("Jobcenter")

        # شناسایی ویژگی‌های اضافی (Modifiers)
        if 'abo' in text:
            category_parts.append("Abo")
        if 'ermäßigt' in text:
            category_parts.append("ermäßigt")

        # اگر فقط نام شهر بود و هیچ نوع گروهی شناسایی نشد
        if not category_parts:
            return f"{found_city} - Allgemein"

        # ترکیب بخش‌ها برای ساخت خروجی نهایی
        return f"{found_city} - {' '.join(sorted(list(set(category_parts))))}"

    # قانون بعدی: تعرفه‌ها و قیمت‌های عمومی (اگر مکان پیدا نشد)
    if re.search(r'\d', text) and ('€' in text or 'normal' in text or re.search(r'\d[tT]', text)):
        return "Tarif / Preis"

    # قانون بعدی: گروه‌های عمومی بدون مکان
    if 'kita' in text:
        return "Allgemein - Kita"
    if 'eltern kind' in text:
        return "Allgemein - Eltern Kind"
    if 'geschwisterkind' in text:
        return "Allgemein - Geschwisterkind"

    # قانون آخر: اگر هیچ‌کدام از قوانین بالا اعمال نشد
    return "Unkategorisiert"


df['GroupName'] = df['GroupName'].astype(str).apply(clean_text_group)

show_unique_values(df, "GroupName" , excel_output="Group_name_cleaned.xlsx")

Unique values in column 'GroupName':
GroupName
Steinfurt - Abo ermäßigt    194774
Allgemein - Kita            192025
Tarif / Preis               177624
Bochum - Schüler            170965
Emsdetten - Abo Schüler     163331
                             ...  
Wersen - Lehrer                  6
Werne - Lehrer                   3
Herten - Lehrer                  1
Oelde - Lehrer                   1
Westerkappeln - Lehrer           1
Name: count, Length: 165, dtype: int64

Results saved to Excel file: Group_name_cleaned.xlsx


In [12]:
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler

categorical_cols = ['MenuName','GroupName','Site','SchoolID']
numerical_cols = ['MenuPrice' , 'MenuSubsidy']
date_cols = ['DateOfService','DateOfOrder']

# ایجاد X و Y
X = df.drop(columns=['OrderQty'], axis=1)
Y = df['OrderQty'].values.ravel()

# ایجاد مپینگ برای ستون‌های کتگوریکال
label_mapping = {}
for col in categorical_cols:
    unique_values = sorted(X[col].unique())
    label_mapping[col] = {val: idx for idx, val in enumerate(unique_values)}

# ایجاد لیست دسته‌بندی‌ها برای OrdinalEncoder
categories_list = []
for col in categorical_cols:
    mapping = label_mapping[col]
    categories = [k for k, v in sorted(mapping.items(), key=lambda item: item[1])]
    categories_list.append(categories)
print(label_mapping)

# Saving the label mapping as a CSV file
mapping_df_list = []

for i, mapping in label_mapping.items():
    temp_df = pd.DataFrame(list(mapping.items()), columns=['Original_Value', 'Encoded_Value'])
    temp_df['column_Name'] = i
    mapping_df_list.append(temp_df)

final_mapping_df = pd.concat(mapping_df_list)
csv_mapping_file_path = 'label_mapping.csv'
final_mapping_df.to_csv(csv_mapping_file_path, index=False)

print(csv_mapping_file_path)



# 1) Encode ستون‌های کتگوریکال
ordinal_encoder = OrdinalEncoder(categories=categories_list)
df[categorical_cols] = ordinal_encoder.fit_transform(df[categorical_cols])





{'MenuName': {'Kiga FZ Nicolai Lippstadt': 0, 'Lunchpaket': 1, 'Lunchpaket vegetarisch': 2, 'Menu A': 3, 'Menu A Ohne Dessert': 4, 'Menu B': 5, 'Menu B1': 6, 'Menu B1 Ohne Dessert': 7, 'Menu B1 veg': 8, 'Menu B2': 9, 'Menu B2 Ohne Dessert': 10, 'Menu C': 11, 'Menu D': 12, 'Menu DGE': 13, 'Mittagessen': 14, 'Mittagessen Outlaw': 15, 'Musterlinie': 16, 'Nudelbar': 17, 'Riesenbeck': 18, 'Salatbar': 19, 'Smart Eating': 20, 'Smart Eating Buffet': 21, 'Tagesmenü': 22, 'veganes Menu': 23}, 'GroupName': {'Allgemein - Kita': 0, 'Arnsberg - ermäßigt': 1, 'Bad Sassendorf - Allgemein': 2, 'Bad Sassendorf - Kita': 3, 'Bad Sassendorf - ermäßigt': 4, 'Beckum - Abo': 5, 'Beckum - Allgemein': 6, 'Beckum - Lehrer': 7, 'Beckum - Schüler': 8, 'Bergkamen - Abo Schüler': 9, 'Bergkamen - Schüler': 10, 'Bielefeld - Abo': 11, 'Bielefeld - Allgemein': 12, 'Bielefeld - Jobcenter': 13, 'Bielefeld - Lehrer': 14, 'Bielefeld - Schüler': 15, 'Bochum - Abo': 16, 'Bochum - Schüler': 17, 'Borghorst - Abo Lehrer': 18, 'B

In [13]:
df.to_csv('clear.csv',index=False, date_format='%Y-%m-%d')

In [ ]:
categorical_cols = ['MenuName','GroupName','Site','SchoolID']


import featuretools as ft

# 1. افزودن یک شناسه منحصر به فرد به دیتافریم اصلی
df['order_id'] = df.index

# 2. ساخت EntitySet
es = ft.EntitySet(id='orders_dataset')

# 3. افزودن دیتافریم با استفاده از متد جدید .add_dataframe
es = es.add_dataframe(
    dataframe_name='orders',
    dataframe=df,
    index='order_id',
    time_index='DateOfOrder'
)

# 4. اجرای Deep Feature Synthesis (DFS) برای ساخت ویژگی‌های جدید
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name='orders',
    trans_primitives=['day', 'month', 'year', 'weekday', 'is_weekend'],
    max_depth= 1,
    verbose=1
)

# 5. ادغام ویژگی‌های جدید به دیتافریم اصلی و حذف ستون‌های غیرضروری
# ابتدا ستون‌هایی که در df اصلی نیستند را جدا می‌کنیم (یعنی ویژگی‌های جدید)
new_feature_columns = feature_matrix.columns.difference(df.columns)

# ویژگی‌های جدید را به دیتافریم اصلی الحاق می‌کنیم
df = df.join(feature_matrix[new_feature_columns])

# در نهایت ستون‌های تاریخ و شناسه موقت را حذف می‌کنیم
df.drop(columns=['DateOfService', 'DateOfOrder', 'DateOfCancel', 'order_id'], inplace=True)

# نمایش نتیجه نهایی
print("ویژگی‌های جدید با موفقیت اضافه شدند.")
print(df.head())
print('ابعاد نهایی دیتافریم:', df.shape)


/home/mojtaba/miniforge3/envs/cuml/lib/python3.12/site-packages/cudf/pandas/fast_slow_proxy.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return fn(*args, **kwargs)
/home/mojtaba/miniforge3/envs/cuml/lib/python3.12/site-packages/cudf/pandas/fast_slow_proxy.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return fn(*args, **kwargs)
/home/mojtaba/miniforge3/envs/cuml/lib/python3.12/site-packages/cudf/pandas/fast_slow_proxy.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return fn(*args, **kwargs)
/home/mojtaba/miniforge3/envs/cuml/lib/python3.12/site-packages/

Built 24 features
Elapsed: 00:20 | Progress:  95%|█████████▍